In [1]:
import torch
import os
import cv2
from tqdm.notebook import tqdm
import cv2 
import torch
import argparse
import os
from torchvision import transforms
import torch.nn as nn
import cv2
import numpy as np
import torch.nn.functional as F
import random
from datetime import datetime
import xlsxwriter
from sklearn.preprocessing import StandardScaler    
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, classification_report
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
#compensate for data imbalance
from sklearn.utils import shuffle
import pickle

In [2]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
meanDataset =[0.4415734549019608,0.388225627197976,0.3462771010120177]
stdDataset =[0.2711501212166684,0.24437697540137235,0.2350548283233844]

In [3]:
#the function you need to use to remove the classifier layer
class Identity(torch.nn.Module):
    def __init__(self):
        super(Identity, self).__init__()
        
    def forward(self, x):
        return x

def init_weights(m):
    if isinstance(m, nn.Conv2d):
        torch.nn.init.kaiming_normal_(m.weight)

def ResizedTensor(image, size): # the function to resize the images to the desired size
    dim=size
    resized = cv2.resize(image, dim, interpolation =  cv2.INTER_LINEAR )
    #print(resized.shape)
    return resized

def CenterCropTensor(image, size): # the function to to do center crop

    image_width, image_height, channel = image.shape

    #image_width, image_height = _get_image_size(img)
    crop_height, crop_width = size

    # crop_top = int(round((image_height - crop_height) / 2.))
    # Result can be different between python func and scripted func
    # Temporary workaround:
    crop_top = int((image_height - crop_height + 1) * 0.5)
    # crop_left = int(round((image_width - crop_width) / 2.))
    # Result can be different between python func and scripted func
    # Temporary workaround:
    crop_left = int((image_width - crop_width + 1) * 0.5)
    img=image[ crop_left: crop_left+crop_width,crop_top:crop_top+crop_height,:];
    return img #crop(img, crop_top, crop_left, crop_height, crop_width)

class BinaryClassification(nn.Module):
        def __init__(self):
            super(BinaryClassification, self).__init__()
            # Number of input features is 12.
            self.layer_1 = nn.Linear(4096, 64) 
            self.layer_2 = nn.Linear(64, 64)
            self.layer_out = nn.Linear(64, 1) 
            
            self.relu = nn.ReLU()
            self.dropout = nn.Dropout(p=0.1)
            self.batchnorm1 = nn.BatchNorm1d(64)
            self.batchnorm2 = nn.BatchNorm1d(64)
            
        def forward(self, inputs):
            x = self.relu(self.layer_1(inputs)) 
            x = self.batchnorm1(x)
            x = self.relu(self.layer_2(x))
            x = self.batchnorm2(x)
            #x = self.dropout(x)
            x = self.layer_out(x) 
            x=torch.sigmoid(x)   
            return x

In [4]:
#step 1 preprocess the 2 images

def image_preprocess(path):
    image=cv2.imread(path)
    masked  =  image / 255 
    for ch in range (0,3):
        masked[:,:,ch]=(masked[:,:,ch]-meanDataset[ch])/stdDataset[ch]  
    masked=ResizedTensor(masked,(256,256))
    masked=CenterCropTensor(masked,(224,224))
    masked=torch.tensor(masked).permute(2, 0, 1).unsqueeze(0).to(device, dtype=torch.float)  #this part move the image from CPU to GPU
    return masked
#step 2 calculate the feature maps of the two images by applying model1 (feature extractor which was trained on the train dataset)
#the function you need to use to remove the classifier layer
class Identity(nn.Module):
    def __init__(self):
        super(Identity, self).__init__()
    def forward(self, x):
        return x
        
def extract_features(classifier_path,image):
    model=torch.load(classifier_path).eval()
    model.classifier[6]=Identity()
    model.fc=Identity()
    model.to(device).eval() #put the model on GPU
    f=model(image).cpu().detach().numpy()
    f_norm=f/np.sqrt(np.dot(f,np.transpose(f)))
    return f_norm
    
#step 3 find the error between the feature maps
def error(f1,f2):
    A=f1-f2
    return A[0]
#step 4 predict if the error relates to the same/ different identitites by applying the classification model on the error vector (which was trained on test dataset)
def same_different(unmasked_image_path,masked_image_path,feature_extractor_path,classifier_path):
    unmasked=image_preprocess(unmasked_image_path)
    masked=image_preprocess(masked_image_path)
    f_unmasked=extract_features(feature_extractor_path,unmasked)
    f_masked=extract_features(feature_extractor_path,masked)
    er=error(f_unmasked,f_masked)

    dic={}
    for kk in range(len(er)):
        dic[str(kk+1)]=er[kk]
    dict={'1':dic}

    ddf=pd.DataFrame(dict).transpose()
    scaler = pickle.load(open('scaler.pkl', 'rb'))
    X_test = scaler.transform(ddf) 

    ## test data    
    class TestData(Dataset):
        
        def __init__(self, X_data):
            self.X_data = X_data
            
        def __getitem__(self, index):
            return self.X_data[index]
            
        def __len__ (self):
            return len(self.X_data)

    

    test_data = TestData(torch.FloatTensor(X_test))
    test_loader = DataLoader(dataset=test_data, batch_size=1)

    classifier=torch.load(classifier_path)
    classifier.to(device).eval()

    with torch.no_grad():
            for X_batch in test_loader:
                X_batch = X_batch.to(device)
                y_test_pred = classifier(X_batch)
                #y_test_pred = torch.sigmoid(y_test_pred)
                y_pred_tag = torch.round(y_test_pred)
                if y_pred_tag.item()==1.0:
                    return 'same'
                else:
                    return 'different'

In [5]:
unmasked_image_path='Amelia_Vega_0002.jpg'
masked_image_path='Wen_Jiabao_0002.jpg_mask.jpg'#'Amelia_Vega_0001.jpg_mask.jpg'
feature_extractor_path='modelWithBestValidationAcc_unmasked_100epochs_v2.h5'
classifier_path='classification_model.h5'
same_different(unmasked_image_path,masked_image_path,feature_extractor_path,classifier_path)

'same'